# Replcation Study -- Myotonic Dystrophy alternative splicing

Checking if our data supports previous evidence about alternative splicing.

## setup

- install Biopython
- install rclone
- setup rclone to point to my google drive (or copy the files directly from https://drive.google.com/open?id=0B0QdpQaEjDtBcF9mNHM5aWEyMzQ) 
- run setup.sh
- install redis and run it on the standard port
- run generateMetadata.ipynb BEFORE running this script.
- run ComputeAS.ipynb AFTER running this script.

In [7]:
# make sure this doesn't clash with anything else!
redisDB = 2

# this will wipe out only the data generated previously by this script and owned by the script (i.e. only db=redisDB)
wipeDB = False

# set this to 2**64 to get everything, or lower value for debugging
recordNo = 10

In [8]:
import redis
r = redis.StrictRedis(host='localhost', port=2050, db=redisDB)


# Remember about 2 weird things:
1. The probe sequence across patients is reverse-sorted (unlike the metadata), because of how redis's lpush works, I might fix that later.

2. Accessing Affy v4 coordinates needs reversing X, Y -> Y, X. I might fix that later.

# Let us hack Biopython's code a little

In [3]:
# Copyright 2004 by Harry Zuzan. All rights reserved.
# Copyright 2016 by Adam Kurkiewicz. All rights reserved.
# This code is part of the Biopython distribution and governed by its
# license.  Please see the LICENSE file that should have been included
# as part of this package.

"""
Reading information from Affymetrix CEL files version 3 and 4.
"""

from __future__ import print_function
import sys
import struct

try:
    import numpy
except ImportError:
    from Bio import MissingPythonDependencyError
    raise MissingPythonDependencyError(
        "Install NumPy if you want to use Bio.Affy.CelFile")


class ParserError(ValueError):
    def __init__(self, *args):
        super(ParserError, self).__init__(*args)

_modeError = ParserError("You're trying to open an Affymetrix v4"
                         " CEL file. You have to use a read binary mode,"
                         " like this `open(filename \"rb\")`.")

# for debugging
# import pprint
# pp = pprint.PrettyPrinter(indent=4)


class Record(object):
    """Stores the information in a cel file

    Example usage:

    >>> from Bio.Affy import CelFile
    >>> with open("Affy/affy_v3_example.CEL", "r") as handle:
    ...     c = CelFile.read(handle)
    ...
    >>> print(c.ncols, c.nrows)
    5 5
    >>> print(c.intensities)
    [[   234.    170.  22177.    164.  22104.]
     [   188.    188.  21871.    168.  21883.]
     [   188.    193.  21455.    198.  21300.]
     [   188.    182.  21438.    188.  20945.]
     [   193.  20370.    174.  20605.    168.]]
    >>> print(c.stdevs)
    [[   24.     34.5  2669.     19.7  3661.2]
     [   29.8    29.8  2795.9    67.9  2792.4]
     [   29.8    88.7  2976.5    62.   2914.5]
     [   29.8    76.2  2759.5    49.2  2762. ]
     [   38.8  2611.8    26.6  2810.7    24.1]]
    >>> print(c.npix)
    [[25 25 25 25 25]
     [25 25 25 25 25]
     [25 25 25 25 25]
     [25 25 25 25 25]
     [25 25 25 25 25]]

    """
    def __init__(self):
        self.version = None
        self.GridCornerUL = None
        self.GridCornerUR = None
        self.GridCornerLR = None
        self.GridCornerLL = None
        self.DatHeader = None
        self.Algorithm = None
        self.AlgorithmParameters = None
        self.NumberCells = None
        self.intensities = None
        self.stdevs = None
        self.npix = None
        self.nrows = None
        self.ncols = None
        self.nmask = None
        self.mask = None
        self.noutliers = None
        self.outliers = None
        self.modified = None


def read(handle):
    """ Reads Affymetrix CEL file and returns Record object.

    CEL files version 3 and 4 are supported, and the parser attempts version detection.

    Example Usage:

    >>> from Bio.Affy import CelFile
    >>> with open("Affy/affy_v4_example.CEL", "rb") as handle:
    ...     c = CelFile.read(handle)
    ...
    >>> c.version == 4
    True
    """
    # If we fail to read the magic number, then it will remain None, and thus
    # we will invoke read_v3 (if mode is not strict), or raise IOError if mode
    # is strict.
    magicNumber = None
    # We check if the handle is a file-like object. If it isn't, and the mode
    # is strict, we raise an error. If it isn't and the mode isn't strict, we
    # continue (perhaps somebody has got a CEL-file-like iterable, which used
    # to work with previous versions of biopython and we don't want to maintain
    # backwards compatibility).
    try:
        mode = handle.mode
        # By definition an Affymetrix v4 CEL file has 64 as the first 4 bytes.
        # Note that we use little-endian irrespective of the platform, again by
        # definition.
        position = handle.tell()
        magicNumber = struct.unpack("<i", handle.read(4))[0]
    except (AttributeError, TypeError):
        pass
    except UnicodeDecodeError:
        raise _modeError
    finally:
        try:
            # reset the offset, to avoid breaking either v3 or v4.
            handle.seek(position)
        except AttributeError:
            pass
    if magicNumber != 64:
        return read_v3(handle)

    else:
        return read_v4(handle)


# read Affymetrix files version 4.
def read_v4(f, headersOnly=False):
    """ Reads Affymetrix CEL file, version 4, and returns a corresponding Record
    object.

    Most importantly record.intensities correspond to intensities from the CEL
    file.

    record.mask and record.outliers are not set.

    Example Usage:

    >>> from Bio.Affy import CelFile
    >>> with open("Affy/affy_v4_example.CEL", "rb") as handle:
    ...     c = CelFile.read_v4(handle)
    ...
    >>> c.version == 4
    True
    >>> print(c.intensities.shape)
    (5, 5)
    """
    # We follow the documentation here:
    # http://www.affymetrix.com/estore/support/developer/powertools/changelog/gcos-agcc/cel.html.affx
    record = Record()
    preHeaders = ["magic", "version", "columns", "rows", "cellNo", "headerLen"]
    preHeadersMap = dict()
    headersMap = dict()

    # load pre-headers
    try:
        for name in preHeaders:
            preHeadersMap[name] = struct.unpack("<i", f.read(4))[0]
    except UnicodeDecodeError as e:
        raise _modeError

    char = f.read(preHeadersMap["headerLen"])
    header = char.decode("ascii", errors = "ignore")
    for header in header.split("\n"):
        if "=" in header:
            header = header.split("=")
            headersMap[header[0]] = "=".join(header[1:])

    # for debugging
    # pp.pprint("preHeadersMap")
    # pp.pprint(preHeadersMap)
    # pp.pprint("headersMap")
    # pp.pprint(headersMap)

    record.version = preHeadersMap["version"]
    if record.version != 4:
        raise ParserError("You are trying to parse CEL file version 4. This"
                         " file violates the structure expected from CEL file"
                         " version 4")
    record.GridCornerUL = headersMap["GridCornerUL"]
    record.GridCornerUR = headersMap["GridCornerUR"]
    record.GridCornerLR = headersMap["GridCornerLR"]
    record.GridCornerLL = headersMap["GridCornerLL"]
    record.DatHeader = headersMap["DatHeader"]
    record.Algorithm = headersMap["Algorithm"]
    record.AlgorithmParameters = headersMap["AlgorithmParameters"]
    record.NumberCells = preHeadersMap["cellNo"]
    # record.intensities are set below
    # record.stdevs are set below
    # record.npix are set below
    record.nrows = int(headersMap["Rows"])
    record.ncols = int(headersMap["Cols"])

    # These cannot be reliably set in v4, because of discrepancies between real
    # data and the documented format.
    record.nmask = None
    record.mask = None
    record.noutliers = None
    record.outliers = None
    record.modified = None

    # Real data never seems to have anything but zeros here, but we don't want
    # to take chances. Raising an error is better than returning unreliable
    # data.
    def raiseBadHeader(field, expected):
        actual = int(headersMap[field])
        message = "The header {field} is expected to be 0, not {value}".format(value=actual, field=field)
        if actual != expected:
            raise ParserError(message)

    raiseBadHeader("Axis-invertX", 0)

    raiseBadHeader("AxisInvertY", 0)

    raiseBadHeader("OffsetX", 0)

    raiseBadHeader("OffsetY", 0)

    # This is unfortunately undocumented, but it turns out that real data has
    # the `record.AlgorithmParameters` repeated in the data section, until an
    # EOF, i.e. b"\x04".
    char = b"\x00"
    safetyValve = 10**4
    for i in range(safetyValve):
        char = f.read(1)
        # For debugging
        # print([i for i in char], end="")
        if char == b"\x04":
            break
        if i == safetyValve:
            raise ParserError("Parse Error. The parser expects a short, "
                              "undocumented binary blob terminating with "
                              "ASCII EOF, x04")

    # After that there are precisely 15 bytes padded. Again, undocumented.
    padding = f.read(15)

    # That's how we pull out the values (triplets of the form float, float,
    # signed short).
    structa = struct.Struct("< f f h")

    # There are 10 bytes in our struct.
    structSize = 10

    # We initialise the most important: intensities, stdevs and npixs.
    record.intensities = numpy.empty(record.NumberCells, dtype=float)
    record.stdevs = numpy.empty(record.NumberCells, dtype=float)
    record.npix = numpy.empty(record.NumberCells, dtype=int)

    b = f.read(structSize * record.NumberCells)
    if not headersOnly:
        for i in range(record.NumberCells):
            binaryFragment = b[i * structSize: (i + 1) * structSize]
            intensity, stdevs, npix = structa.unpack(binaryFragment)
            record.intensities[i] = intensity
            # Turn these off, because we don't care
            #record.stdevs[i] = stdevs
            #record.npix[i] = npix

        # reshape without copying.
        def reshape(array):
            view = array.view()
            view.shape = (record.nrows, record.ncols)
            return view

        record.intensities = reshape(record.intensities)
    return record

# Let us first load the data from the raw files into memory.

In [5]:
from utils import *
import re
import os

CEL_PATH = "CEL_files"

MUSCLE_CELS = [filename for filename in os.listdir(CEL_PATH) if "_M.CEL" in filename and ".swp" not in filename]
#MUSCLE_RERUN_CELS = [filename for filename in os.listdir(CEL_PATH) if "_MR.cel" in filename and ".swp" not in filename]
#BLOOD_CELS = [filename for filename in os.listdir(CEL_PATH) if "_B.CEL" in filename and ".swp" not in filename]

#Let's not load muscle and blood CELs for now.
MUSCLE_RERUN_CELS = []
BLOOD_CELS = []

ALL_CELS = MUSCLE_CELS + MUSCLE_RERUN_CELS + BLOOD_CELS

#assertEqual(len(ALL_CELS), 76)
#assertEqual(set(MUSCLE_CELS).intersection(set(MUSCLE_RERUN_CELS)), set())

ALL_RECORDS = {}

for filename in ALL_CELS:
    with open(os.path.join(CEL_PATH, filename), "rb") as f:
        record = read_v4(f, headersOnly=True)
        ALL_RECORDS[filename] = record
        record.npix = None
        record.stdevs = None

# Then, let us extract HUEX IDs from DatHeader

In [6]:
FILE_TO_HUEX = {}

for filename, record in ALL_RECORDS.items():
    huexGroup = re.search("(\d)*HUEX1A[\d]*", record.DatHeader)
    if huexGroup:
        huex = huexGroup.group()
        FILE_TO_HUEX[filename] = huex

# Then let us map HUEX IDs onto allele length

In [7]:
ALLELE_PATH = "alleleLength"

HUEX_TO_MOD = {}

HUEX_TO_EST = {}

BAD_HUEX = set()

with open(ALLELE_PATH) as f:
    for line in f:
        line = line.rstrip().split("\t")
        header = line[0]
        ender = line[1:]
        if header == "IDs":
            IDs = ender
        if header == "EstProgAllele":
            EstProgAllele = ender
        if header == "ModalAllele":
            ModalAllele = ender
    for id, est, mod in zip(IDs, EstProgAllele, ModalAllele):
        count = False
        try:
            HUEX_TO_MOD[id] = int(mod)
        except ValueError:
            HUEX_TO_MOD[id] = None
            BAD_HUEX.add(id)
        try:
            HUEX_TO_EST[id] = int(est)
        except ValueError:
            HUEX_TO_MOD[id] = None
            BAD_HUEX.add(id)

In [8]:
HUEX_TO_FILE_MUSCLE = {FILE_TO_HUEX[key] : key for key in MUSCLE_CELS}
FILE_MUSCLE_TO_HUEX = {value: key for key, value in HUEX_TO_FILE_MUSCLE.items()}
allMetadata = sorted([(HUEX_TO_MOD[huex], HUEX_TO_EST[huex],  huex, filename) for huex, filename in HUEX_TO_FILE_MUSCLE.items() if huex not in BAD_HUEX])

In [9]:
assertEqual(len(BAD_HUEX), 1)

In [10]:
assertEqual(len(MUSCLE_CELS), len(HUEX_TO_FILE_MUSCLE))
assertEqual(len(HUEX_TO_FILE_MUSCLE), len(FILE_MUSCLE_TO_HUEX))

In [11]:
print(HUEX_TO_FILE_MUSCLE)

{'190281HUEX1A11': '427374914_M.CEL', '189749HUEX1A11': '117440822_M.CEL', '190283HUEX1A11': '549452228_M.CEL', '189740HUEX1A11': '159834720_M.CEL', '189737HUEX1A11': '873750289_M.CEL', '190285HUEX1A11': '661252781_M.CEL', '189741HUEX1A11': '270148799_M.CEL', '189738HUEX1A11': '830225708_M.CEL', '190278HUEX1A21': '204472077_M.CEL', '190287HUEX1A11': '896445336_M.CEL', '189819HUEX1A11': '881676366_M.CEL', '189745HUEX1A11': '406335477_M.CEL', '189822HUEX1A11': '360448352_M.CEL', '189747HUEX1A11': '572448109_M.CEL', '190280HUEX1A11': '420299717_M.CEL', '189823HUEX1A11': '111747589_M.CEL', '189748HUEX1A11': '597785396_M.CEL', '189746HUEX1A11': '449599671_M.CEL', '190286HUEX1A11': '819054051_M.CEL', '189751HUEX1A11': '387939296_M.CEL', '189743HUEX1A11': '321962190_M.CEL', '190284HUEX1A11': '575039926_M.CEL', '189821HUEX1A11': '419550533_M.CEL', '189744HUEX1A11': '328687703_M.CEL', '189820HUEX1A11': '377666471_M.CEL', '189739HUEX1A11': '129523253_M.CEL', '190279HUEX1A11': '230974357_M.CEL', 

# It turns out we have all the IDs we want, and we know allele lengths for all the patients except one.

The patient refused blood donation after the 3rd unsucessful attempt.

# Now, let us look at the CDF file

The CDF file contains information mapping transcription clusters onto physical coordinates on the chip.

In [12]:
import sys
contextDict = {}

with open(os.path.join(CEL_PATH, "HuEx-1_0-st-v2.text.cdf")) as f:
    lines = f.readlines()
    context = None
    for line in lines:
        line = line.rstrip()
        if line:
            c1 = line[0] == "["
            c2 = "Unit" in line
            c3 = "_Block1]" in line
            checkConds = [c1, c2, c3]
            if all(checkConds):
                context = line[len("Unit") + 1: -1 * len("_Block1") - 1]
                if context in contextDict:
                    print(line)
                    raise ValueError
                else:
                    contextDict[context] = {}
            elif context is not None:
                eqPos = line.find("=")
                key = line[:eqPos]
                matchedList = re.findall("Cell[1-9]+", key)
                if matchedList:
                    value = line[eqPos + 1:].split("\t")
                    contextDict[context][key] = value[:3]
        else:
            context = None

In [23]:
def extractIntensity(record, coord):
    # It is important to reverse the coordinates from Row/Column into X/Y
    return record.intensities[coord[1], coord[0]]

In [28]:
import redis
import json

allMetadata.sort()
    
if wipeDB:
    r.flushdb()

r.set("main$alleleData", json.dumps(allMetadata))

def upsertProbes(probeset, probes):
    r.hset("probes$metadata", probeset, json.dumps(probes))

def appendData(probeset, probes, record):
    record = record[0]
    intensities = []
    for _, X, Y in probes:
        intens = extractIntensity(record, [X, Y])
        intensities.append(int(intens))
    r.lpush("probes$probeset$" + probeset, json.dumps(intensities))

def iterateOverContext(upperLimit, callable, extraArgs = None):
    gen = contextDict.items()
    for i, (probeset, XYseq) in enumerate(gen):
        if i == recordNo:
            break
        probes = []
        for cell, (X, Y, Seq) in XYseq.items():
            probes.append((Seq, int(X), int(Y)))
        probes.sort()
        if extraArgs:
            callable(probeset, probes, extraArgs)
        else:
            callable(probeset, probes)

# That's the dangerous and time-consuming bit, where we iterate over all intensities of all patients and put them in the database

In [29]:
iterateOverContext(recordNo, upsertProbes)
for modAllele, progAllele, huexID, filename in allMetadata:
    with open(os.path.join(CEL_PATH, filename), "rb") as f:
        record = CELFile.read_v4(f)
        record.npix = None
        record.stdevs = None
        iterateOverContext(recordNo, appendData, extraArgs=[record])
r.save()

b'[[11, 11, "189822HUEX1A11", "360448352_M.CEL"], [12, 12, "189747HUEX1A11", "572448109_M.CEL"], [12, 12, "189748HUEX1A11", "597785396_M.CEL"], [12, 12, "189751HUEX1A11", "387939296_M.CEL"], [83, 80, "190284HUEX1A11", "575039926_M.CEL"], [186, 158, "190280HUEX1A11", "420299717_M.CEL"], [240, 199, "189737HUEX1A11", "873750289_M.CEL"], [261, 155, "189820HUEX1A11", "377666471_M.CEL"], [290, 186, "189743HUEX1A11", "321962190_M.CEL"], [297, 227, "189738HUEX1A11", "830225708_M.CEL"], [297, 227, "189749HUEX1A11", "117440822_M.CEL"], [345, 243, "190282HUEX1A11", "473208969_M.CEL"], [373, 243, "190278HUEX1A21", "204472077_M.CEL"], [408, 211, "189739HUEX1A11", "129523253_M.CEL"], [561, 411, "190286HUEX1A11", "819054051_M.CEL"], [571, 297, "189745HUEX1A11", "406335477_M.CEL"], [593, 301, "189750HUEX1A11", "124563003_M.CEL"], [604, 439, "190283HUEX1A11", "549452228_M.CEL"], [654, 318, "189744HUEX1A11", "328687703_M.CEL"], [697, 397, "190287HUEX1A11", "896445336_M.CEL"], [740, 506, "189741HUEX1A11"

In [9]:
r.hget("probes$metadata", 3973794)

b'[["AACAAGGGACACTCATTGCGGGAAC", 2518, 1946], ["ACGACGTAACGATCTGACGGAAAAG", 1364, 263], ["CAGTGGTCCAAACGATACAATACAT", 1802, 1380], ["CTCGGTAGGAAAGAACCCCGTATGT", 1783, 123]]'

In [11]:
address = "probes$probeset$" + "2346130"
print(address)
length = r.llen(address)
for i in range(length):
    print(i, r.lindex(address, i))

probes$probeset$2346130
0 b'[66, 81, 106, 90]'
1 b'[61, 66, 94, 94]'
2 b'[61, 39, 140, 47]'
3 b'[49, 47, 82, 56]'
4 b'[82, 69, 136, 72]'
5 b'[36, 46, 134, 63]'
6 b'[77, 72, 87, 68]'
7 b'[59, 59, 125, 75]'
8 b'[67, 70, 110, 86]'
9 b'[100, 85, 118, 73]'
10 b'[94, 84, 100, 74]'
11 b'[66, 49, 66, 68]'
12 b'[40, 44, 36, 50]'
13 b'[42, 56, 76, 51]'
14 b'[95, 75, 85, 65]'
15 b'[75, 57, 138, 73]'
16 b'[49, 58, 77, 50]'
17 b'[38, 58, 53, 47]'
18 b'[45, 35, 64, 59]'
19 b'[56, 68, 89, 66]'
20 b'[63, 53, 96, 87]'
21 b'[64, 46, 94, 42]'
22 b'[72, 72, 131, 59]'
23 b'[58, 82, 96, 54]'
24 b'[53, 34, 60, 64]'
25 b'[35, 38, 61, 46]'
26 b'[62, 59, 86, 52]'
27 b'[73, 61, 113, 70]'
28 b'[48, 37, 60, 43]'
